In [1]:
import numpy as np
import onsager.PowerExpansion as PE

In [17]:
t3d = PE.Taylor3D()

In [19]:
t3d.Ylm2ind

array([[ 0, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 2,  3, -1, -1, -1, -1, -1, -1,  1],
       [ 6,  7,  8, -1, -1, -1, -1,  4,  5],
       [12, 13, 14, 15, -1, -1,  9, 10, 11],
       [20, 21, 22, 23, 24, 16, 17, 18, 19]])

In [112]:
projLylm=np.zeros((t3d.Lmax+2,t3d.NYlm,t3d.NYlm),dtype=complex)

In [113]:
for l,m in ((l,m)for l in range(t3d.Lmax+1)for m in range(-l,l+1)):
    lm = t3d.Ylm2ind[l,m]
    projLylm[l,lm,lm] = 1
    projLylm[-1,lm,lm] = 1

In [114]:
projLylm[2].shape

(25, 25)

In [115]:
#Now we check that the number of 1+0js on the diagonals is correct.
#For l=2, there should be 5. In general there should be 2l+1
counts = np.zeros(t3d.Lmax+1)
for l in range(t3d.Lmax+1):
    for i in range(25):
        if projLylm[l,i,i] == 1+0j:
            counts[l]+=1
print(counts)

[1. 3. 5. 7. 9.]


In [116]:
#now, we work out the inner tensordot, for say l=2
tdot = np.tensordot(projLylm[2],t3d.powYlm,axes=(1,1))
tdot_test = np.zeros((projLylm[2].shape[0],t3d.powYlm.shape[0]),dtype=complex) #th surviving axes
for i in range(projLylm[2].shape[0]):#We eliminate axis 1
    for j in range(t3d.powYlm.shape[0]):#We eliminate axis 1
        for k in range(t3d.powYlm.shape[1]):
            tdot_test[i,j] += projLylm[2][i,k]*t3d.powYlm[j,k]
np.allclose(tdot,tdot_test)

True

In [117]:
#Now, let us do the same thing for all ls
tdot_test = np.zeros((t3d.Lmax+2,t3d.NYlm,t3d.powYlm.shape[0]),dtype=complex) #th surviving axes
for l in range(t3d.Lmax+2):
    for i in range(projLylm[l].shape[0]):#We eliminate axis 1
        for j in range(t3d.powYlm.shape[0]):#We eliminate axis 1
            for k in range(t3d.powYlm.shape[1]):
                tdot_test[l][i,j] += projLylm[l][i,k]*t3d.powYlm[j,k]

In [118]:
tdot_test[0][0][0]

(3.5449077018110318+0j)

In [119]:
projLylm[0][0]

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j])

In [120]:
t3d.ind2pow[10]

array([0, 0, 3])

In [121]:
for i,j in enumerate(t3d.powYlm[10]):
    print(i,j)

0 0j
1 0j
2 (1.2279920495357857+0j)
3 0j
4 0j
5 0j
6 0j
7 0j
8 0j
9 0j
10 0j
11 0j
12 (0.5359396685525429+0j)
13 0j
14 0j
15 0j
16 0j
17 0j
18 0j
19 0j
20 0j
21 0j
22 0j
23 0j
24 0j


In [122]:
t3d.ind2Ylm[2],t3d.ind2Ylm[12]

(array([1, 0]), array([3, 0]))

In [123]:
for i in range(25):
    print(i,projLylm[3][i,i])

0 0j
1 0j
2 0j
3 0j
4 0j
5 0j
6 0j
7 0j
8 0j
9 (1+0j)
10 (1+0j)
11 (1+0j)
12 (1+0j)
13 (1+0j)
14 (1+0j)
15 (1+0j)
16 0j
17 0j
18 0j
19 0j
20 0j
21 0j
22 0j
23 0j
24 0j


In [124]:
tdot_test[3][12][10]

(0.5359396685525429+0j)

In [125]:
#So, the inner tensor product, tdot[l][i][j] gives the contribution of the i_th [l,m] combination in ind2Ylm,
#to the expansion of the j_th (n0+n1+n2=l) combination in ind2pow

In [126]:
#So, now let's do the outer tensor product
projL = np.zeros((t3d.Lmax + 2, t3d.Npower, t3d.Npower))
for l in range(t3d.Lmax+2):
    projL[l]=np.tensordot(t3d.Ylmpow,tdot_test[l],axes=(0,0)).real

In [127]:
np.allclose(projL,t3d.Lproj)

True

In [130]:
t3d.Ylmpow.shape,tdot_test[0].shape

((25, 35), (25, 35))

In [137]:
#We will be eliminating the 0th axes. So axes 1 survive.
tdot_outer=np.zeros((t3d.Lmax+2,t3d.Npower,t3d.Npower))
for l in range(t3d.Lmax+2):
    for i in range(t3d.Ylmpow.shape[1]):#the surviving axes of the first arguments of the tensordot
        for j in range(tdot_test[l].shape[1]):
            for k in range(tdot_test[l].shape[0]):
                tdot_outer[l][i,j] += (t3d.Ylmpow[k,i]*tdot_test[l][k,j]).real
        

In [138]:
np.allclose(t3d.Lproj,tdot_outer.real)

True

In [139]:
tdot_outer.shape

(6, 35, 35)

In [140]:
t3d.Lproj.shape

(6, 35, 35)